In [1]:
class Sentiment:
    NEGATIVE='Negative'
    POSITIVE='Positve'
    NEUTRAL='Neutral'

In [2]:
import random
class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
        
    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.NEGATIVE
        elif self.score==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
    
    def get_text(self):
        return[x.text for x in self.reviews]
    
    def get_sentiment(self):
        return[y.sentiment for y in self.reviews]
    
    def evenly_distribute(self):
        negative=list(filter(lambda x: x.sentiment==Sentiment.NEGATIVE, self.reviews))
        positive=list(filter(lambda x: x.sentiment==Sentiment.POSITIVE, self.reviews))
        positive_shrunk=positive[:len(negative)]
        self.reviews=positive_shrunk+negative
        random.shuffle(self.reviews)

In [3]:
import json

In [4]:
file_name="Books_small_10000.json"

In [5]:
reviews=[]
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))

In [6]:
reviews[5].sentiment

'Positve'

Prep Data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
training,test=train_test_split(reviews,test_size=0.33, random_state=42)
train_container=ReviewContainer(training)
test_container=ReviewContainer(test)


In [9]:
train_container.evenly_distribute()
X_train=train_container.get_text()
y_train=train_container.get_sentiment()

print(y_train.count(Sentiment.NEGATIVE))
print(y_train.count(Sentiment.POSITIVE))

436
436


In [10]:
test_container.evenly_distribute()
X_test=test_container.get_text()
y_test=test_container.get_sentiment()
print(y_test.count(Sentiment.NEGATIVE))
print(y_test.count(Sentiment.POSITIVE))

208
208


In [11]:
y_train[5]


'Negative'

# Bag of Words Vectorizer

In [22]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer=TfidfVectorizer()

In [23]:
X_train_Vectors=vectorizer.fit_transform(X_train)
X_test_Vectors=vectorizer.transform(X_test)

# # using classifiers

In [24]:
from sklearn import svm
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(X_train_Vectors,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [25]:
print(X_test[55])
print(test[55].score)
clf_svm.predict(X_test_Vectors[55])

This was such a great story. I love the characters and couldn't wait to see what the next chapter had to offer. I would recommend this story to all of my friends and family! Get your copy today
5.0


array(['Positve'], dtype='<U8')

In [26]:
#for i in range(100):
#    print(f'{i}{test[i].sentiment}')

In [27]:
from sklearn.metrics import f1_score

In [28]:
f1_score(y_test, clf_svm.predict(X_test_Vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

In [19]:
y_test.count(Sentiment.POSITIVE)

208

In [20]:
test_model=['this is good book', 'this is really bad book', "don't buy this book please",'i am fun']
test_model_vectorizer=vectorizer.transform(test_model)

In [21]:
clf_svm.predict(test_model_vectorizer)

array(['Positve', 'Negative', 'Negative', 'Positve'], dtype='<U8')

#Tuning of Model

In [29]:
from sklearn.model_selection import GridSearchCV

In [32]:
parameter={'kernel':('linear','rbf'),'C':(1,4,8,16)}
svc=svm.SVC()
clf=GridSearchCV(svc, parameter, cv=5)
clf.fit(X_test_Vectors,y_train)


ValueError: Found input variables with inconsistent numbers of samples: [416, 872]